In [5]:
import keras
from keras.datasets import mnist
from keras import backend as K

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau

In [6]:
num_classes = 10
epochs = 15
batch_size = 64

img_rows, img_cols = 28, 28
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)
    
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)


In [11]:
print(x_train[0].shape)

(28, 28, 1)


In [3]:
classifier = Sequential()

classifier.add(Conv2D(64, (2, 2), input_shape = (28, 28, 1), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(64, (2, 2), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (1, 1)))
classifier.add(Dropout(0.2))

classifier.add(Flatten())

classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units = 10, activation = 'softmax'))

classifier.compile(optimizer = keras.optimizers.Adadelta(), loss = 'categorical_crossentropy', metrics = ['accuracy'])

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
classifier.fit(x_train, y_train, epochs = epochs, verbose = 1, validation_data = (x_test, y_test))
score = classifier.evaluate(x_test, y_test, verbose=0)
print('Test accuracy:', score[1])

Instructions for updating:
Use tf.cast instead.
Train on 60000 samples, validate on 10000 samples
Epoch 1/15
60000/60000 [==============================] - 97s 2ms/step - loss: 0.2272 - acc: 0.9311 - val_loss: 0.0659 - val_acc: 0.9797
Epoch 2/15
53632/60000 [=========================>....] - ETA: 10s - loss: 0.0911 - acc: 0.9734

KeyboardInterrupt: 

In [10]:
classifier.save('mnist.h5')

## load and test

In [1]:
from keras.models import load_model
mnist_model = load_model('mnist.h5')

G:\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import cv2
import numpy as np
import os

In [6]:
def test_all(path):
    sum = 0
    files = [os.path.join(path, x) for x in os.listdir(path) if x.endswith(".png")]
    for imgfile in files:
        img = cv2.imread(imgfile, cv2.IMREAD_GRAYSCALE)
        img = img.astype('float32') / 255.
        img = cv2.resize(img, (28, 28))
        img = img.reshape((1, 28, 28, 1))
        prediction = np.argmax(mnist_model.predict([img]))
        print(os.path.split(imgfile)[1], prediction)
        sum += prediction
    return sum
test_all("./res/")

1.png 2
10.png 3
11.png 1
12.png 2
2.png 3
3.png 3
4.png 5
5.png 9
6.png 5
7.png 3
8.png 4
9.png 4


44